In [1]:

import yaml
import sagemaker
import boto3
import pandas as pd
import os
import json
from sklearn.model_selection import train_test_split
from time import gmtime, strftime
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.predictor import Predictor
from sagemaker import Model

In [2]:

SETTING_FILE_PATH = "../config/settings.yaml"
DATA_FOLDER_PATH = "../avazu-ctr-prediction"

# AWS リソース設定
with open(SETTING_FILE_PATH) as file:
    aws_info = yaml.safe_load(file)
        
sess = sagemaker.Session()

role = aws_info['aws']['sagemaker']['role']
bucket = aws_info['aws']['sagemaker']['s3bucket']
region = aws_info['aws']['sagemaker']['region']

In [10]:
! touch dymmy_model | tar -czf dummy_model.tar.gz dummy_model

In [12]:
endpoint_name = 'latency-check-endpoint'
prefix = 'latency_check'
model_file = "dummy_model.tar.gz"

s3_resource_session = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_session.Object(os.path.join(prefix, "model", model_file)).upload_file(model_file)

In [13]:
model_data = f"s3://{bucket}/{prefix}/model/{model_file}"
model = SKLearnModel(model_data=model_data,
                     role=role, 
                     entry_point=endpoint_name,
                     framework_version="0.23-1",
                     py_version="py3",
                     source_dir="model",
                     entry_point="inference.py",
                     sagemaker_session=sess,)
                    


ValueError: framework_version or py_version was None, yet image_uri was also None. Either specify both framework_version and py_version, or specify image_uri.